In [ ]:
import pandas as pd         # """多機版---20190507"""
import numpy as np
import time
import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima_model import ARIMA
import os
import shutil
import warnings
warnings.filterwarnings('ignore')
import sys
sys.setrecursionlimit(50000)

print('鋼網印刷品質監控中，請勿關閉程序！')

def diff_data(data):
    p=adfuller(data[u'NAME'],1)[1]
    q=float(acorr_ljungbox(data,lags=1)[1])   
    D_data=data.diff().dropna()
    D_data.columns=[u'NAME']
    D_data.dropna(inplace=True)
    t=0
    while  p>0.05 or q>0.05:
        D_data=D_data.diff().dropna()
        D_data.columns=[u'NAME']
        D_data.dropna(inplace=True)
        p=adfuller(D_data[u'NAME'],1)[1]
        q=float(acorr_ljungbox(D_data,lags=1)[1]) 
        t=t+1            
    return p,q,t
#---------------------------------------------------------------------------------------------------
def mkdir(path): 
    folder = os.path.exists(path) 
    if not folder:                   #判?是否存在文件?如果不存在??建?文件?
        os.makedirs(path)            #makedirs ?建文件?如果路?不存在??建??路?
        print ("創建新文件夾:%s"%path)
    else:
        for info in os.listdir(path):
            domain=os.path.abspath(path)  #?取文件?的路?
            info=os.path.join(domain,info) 
            os.remove(info)
            
f=os.listdir('D:\\SPI')            
for i1 in range(len(f)):   
    file1 = "D:\\SPI\\%s\\Real"%f[i1]
    mkdir(file1)
    file2 = "D:\\SPI\\%s\\Caculate"%f[i1]
    mkdir(file2)
#-----------------------------------------------------------------------------------------------------
batch=30
N=30
f=os.listdir('D:\\SPI')
while 1==1:
    for i in range(len(f)):
        if not os.listdir('D:\\SPI\\%s\\SPIDATA'%f[i]):
            print('NO File to Do')
            continue
        else:
            for info1 in os.listdir(r'D:\\SPI\\%s\\Model'%f[i]):       
                a=info1.split('.')[0]        
                for info2 in os.listdir(r'D:\\SPI\\%s\\SPIDATA'%f[i]):    
                    b=info2.split('_')[0]            
                    if a==b:                
                        c=a
                        break
                    else:
                        continue
            if os.path.exists('D:\\SPI\\%s\\Model\\%s.xlsx'%(f[i],c)):
                path=pd.read_excel('D:\\SPI\\%s\\Model\\%s.xlsx'%(f[i],c))
            elif os.path.exists('D:\\SPI\\%s\\Model\\%s.xls'%(f[i],c)):
                path=pd.read_excel('D:\\SPI\\%s\\Model\\%s.xls'%(f[i],c))
            elif os.path.exists('D:\\SPI\\%s\\Model\\%s.csv'%(f[i],c)):
                path=pd.read_excel('D:\\SPI\\%s\\Model\\%s.csv'%(f[i],c))
            pad=path.PAD
            upA=path.upA    
            downA=path.downA   
            upH=path.upH
            downH=path.downH   
        
#---------------------------------------------------------------------------------------------------------------        
        if not os.listdir('D:\\SPI\\%s\\SPIDATA'%f[i]) or len([name for name in os.listdir(r'D:\\SPI\\%s\\SPIDATA'%f[i])])<batch:
            continue   #當文件個數小于batch時，跳?到下一?文件?
        else:   
            file1 = "D:\\SPI\\%s\\Real"%f[i]        
            file2 = "D:\\SPI\\%s\\Caculate"%f[i]     
            domain=os.path.abspath(r'D:\\SPI\\%s\\SPIDATA'%f[i])  #?取文件?的路?
            for info in os.listdir(r'D:\\SPI\\%s\\SPIDATA'%f[i]):
                info=os.path.join(domain,info)    #?路?与文件名?合起?就是每?文件的完整路?
                infoo=open(info,'r')    #?取文件?容 
                d=infoo.readlines()
                for j in range(len(pad)):  
                    DT=[]
                    PAD=[]
                    HIGHT=[]
                    AREA=[]
                    for ii in range(len(d)):
                        if d[ii].split(',')[1]==pad[j]:
                            dt=d[ii].split(',')[0]
                            dt=datetime.datetime.strptime(dt,'%Y/%m/%d %H:%M:%S')
                            DT.append(dt)
                            PAD.append(d[ii].split(',')[1])                        
                            area=int(d[ii].split(',')[2])
                            hight=int(d[ii].split(',')[3].split('\n')[0])            
                            HIGHT.append(hight)                        
                            AREA.append(area)
                        else:
                            continue
                    A=[DT,PAD,AREA,HIGHT]
                    db=pd.DataFrame(A,index=['DateTime','Pad','HIGHT','AREA'])
                    db=db.T
                    db=db.drop_duplicates()  #移除重複值
                    db.sort_values(by='DateTime',ascending=True,inplace=True)  #按升序排序數據，並覆蓋掉原來的

                    for s in range(len(db)):  # 面積標準化 & 清洗異常值                                    
                        if (db.AREA[s]/np.mean(db.AREA)>2) or (db.AREA[s]/np.mean(db.AREA)<0.5):
                            db.AREA[s]=np.mean(db.AREA)
                            db.HIGHT[s]=np.mean(db.HIGHT)                   

                    fileName1='%s.txt'%pad[j]    #把每一個pad點的高度和面積存到對應的文件夾中
                    filePath1=file1+os.path.sep+fileName1
                    db.to_csv(filePath1,sep=',',index=False,header=None,mode='a+')  #['DateTime','Pad','HIGHT','AREA']
                infoo.close()  #關閉已打開的文件 
                os.remove(info)
#--------------------------------------------------------------------------------------------------------------------------                         
            for jj in  range(len(pad)):    #讀取文件夾下所有點的數據計算CPK     
                fileName2='%s.txt'%pad[jj]
                filePath2=file1+os.path.sep+fileName2
                with open(filePath2, 'r') as ff:
                    d2=ff.readlines()
                    DateTime=[]
                    Pad=[]
                    HIGHT=[]
                    AREA=[]
                    for ii in range(len(d2)):
                        dt=d2[ii].split(',')[0]
                        ddt=datetime.datetime.strptime(dt,'%Y-%m-%d %H:%M:%S')
                        DateTime.append(ddt)            
                        Pad.append(d2[ii].split(',')[1])
                        hight=int(d2[ii].split(',')[2])
                        area=int(d2[ii].split(',')[3])   #.split('\n')[0]                               
                        HIGHT.append(hight)
                        AREA.append(area)
                    dd=[DateTime,Pad,HIGHT,AREA]
                    dd=pd.DataFrame(dd,index=['DateTime','Pad','HIGHT','AREA'])
                    db=dd.T 
                    db=db.drop_duplicates()
                    db.sort_values(by='DateTime',ascending=True,inplace=True)

                    upspcH=list(np.zeros(len(pad)))
                    downspcH=list(np.zeros(len(pad)))
                    upspcA=list(np.zeros(len(pad)))
                    downspcA=list(np.zeros(len(pad)))
                    if len(db)<N*batch:   #前面30個batch內的數據根據現有的全部數據計算規格上下限             
                        upspcH[jj]=np.mean(db.HIGHT)+upH[jj]*np.std(db.HIGHT)
                        downspcH[jj]=np.mean(db.HIGHT)-downH[jj]*np.std(db.HIGHT)
                        upspcA[jj]=np.mean(db.AREA)+upA[jj]*np.std(db.AREA)
                        downspcA[jj]=np.mean(db.AREA)-downA[jj]*np.std(db.AREA)
                    else:                 #預測部分使用前面3個batch內的數據算規格上下限
                        upspcH[jj]=np.mean(db.HIGHT[0:N*batch])+upH[jj]*np.std(db.HIGHT[0:N*batch])
                        downspcH[jj]=np.mean(db.HIGHT[0:N*batch])-downH[jj]*np.std(db.HIGHT[0:N*batch])
                        upspcA[jj]=np.mean(db.AREA[0:N*batch])+upA[jj]*np.std(db.AREA[0:N*batch])
                        downspcA[jj]=np.mean(db.AREA[0:N*batch])-downA[jj]*np.std(db.AREA[0:N*batch])  

                    CPKH=[]
                    CPKA=[] 
                    TIME=[]
                    preAREA=[]
                    preHIGHT=[]
                    preTIME=[]
                    numlen=int(len(db)/batch)                
                    for k in range(numlen):    
                        cpkH=min((upspcH[jj]-np.mean(db.HIGHT[k*batch:(batch-1+batch*k)]))/(3*np.std(db.HIGHT[k*batch:(batch-1+batch*k)])),\
                               (np.mean(db.HIGHT[k*batch:(batch-1+batch*k)])-downspcH[jj])/(3*np.std(db.HIGHT[k*batch:(batch-1+batch*k)])))
                        cpkA=min((upspcA[jj]-np.mean(db.AREA[k*batch:(batch-1+batch*k)]))/(3*np.std(db.AREA[k*batch:(batch-1+batch*k)])),\
                               (np.mean(db.AREA[k*batch:(batch-1+batch*k)])-downspcA[jj])/(3*np.std(db.AREA[k*batch:(batch-1+batch*k)])))
                        Time=db.DateTime[k*batch+batch-1]
                        d11=datetime.datetime.timestamp(Time)
                        dateArray=time.localtime(d11)
                        otherStyleTime =time.strftime('%Y-%m-%d %H:%M:%S',dateArray)   #dateArray.strftime("%Y-%m-%d %H:%M:%S")
                        CPKH.append(cpkH)
                        CPKA.append(cpkA)  
                        TIME.append(otherStyleTime)  
                    tha=[TIME,CPKH,CPKA]
                    THA=pd.DataFrame(tha,index=['DateTime','CPKH','CPKA'])
                    THA=THA.T
                    THA=THA.drop_duplicates()
                    THA.sort_values(by='DateTime',inplace=True)

                    fileName3='%s.txt'%pad[jj]
                    filePath3=file2+os.path.sep+fileName3
                    THA.to_csv(filePath3,sep=',',index=False,header=None)  #,mode='a+'
    #----------------------------------------------------------------------------------------------------------------#                   
                    with open(filePath3, 'r') as ff:
                        d2=ff.readlines()                                       
                        if len(d2)<N:
                            continue   # print("數據量小於%s組，暫時達不到建模條件"%N)        
                        else:  
                            if len(d2)>50:
                                d2=d2[(len(d2)-50):len(d2)]
                            else:
                                d2=d2
                            TIME=[]
                            CPKH=[]
                            CPKA=[]  
                            for ii in range(len(d2)):
                                dt=d2[ii].split(',')[0]
                                ddt=datetime.datetime.strptime(dt,'%Y-%m-%d %H:%M:%S')
                                TIME.append(ddt) 
                                cpkh=float(d2[ii].split(',')[1])
                                cpka=float(d2[ii].split(',')[2])
                                CPKH.append(cpkh)
                                CPKA.append(cpka)
                            tha=[TIME,CPKH,CPKA]
                            THA1=pd.DataFrame(tha,index=['TIME','CPKH','CPKA'])
                            THA1=THA1.T
                            THA1=THA1.drop_duplicates()
                            THA1.sort_values(by='TIME',ascending=True,inplace=True)

                            dataA=list(THA1.CPKA)
                            dataH=list(THA1.CPKH)
                            dataA=pd.DataFrame(dataA,columns=['NAME'])
                            dataA.dropna(inplace=True)                
                            dataH=pd.DataFrame(dataH,columns=['NAME'])  
                            dataH.dropna(inplace=True)                    

                            pmax=3
                            qmax=3 
                            dA=diff_data(dataA)[2]
                            dH=diff_data(dataH)[2]    
                            bic_matrixA=[]
                            for pA in range(pmax+1):
                                temp=[]
                                for qA in range(qmax+1):
                                    try:
                                        temp.append(ARIMA(dataA,(pA,dA,qA)).fit().bic) #start_params=(0,0,0)
                                    except:
                                        temp.append(None)
                                bic_matrixA.append(temp)
                            bic_matrixA=pd.DataFrame(bic_matrixA)
                            pA,qA=bic_matrixA.stack().astype('float64').idxmin()
                #             print(u'BIC最小的p值和q值為：,%s,%s'%(pA,qA))
                            modelA=ARIMA(dataA,(pA,dA,qA)).fit()

                            bic_matrixH=[]
                            for pH in range(pmax+1):
                                temp=[]
                                for qH in range(qmax+1):
                                    try:
                                        temp.append(ARIMA(dataH,(pH,dH,qH)).fit().bic)  #start_params=(0,0,0)
                                    except:
                                        temp.append(None)
                                bic_matrixH.append(temp)
                            bic_matrixH=pd.DataFrame(bic_matrixH)
                            pH,qH=bic_matrixH.stack().astype('float64').idxmin()
                #             print(u'BIC最小的p值和q值為：,%s,%s'%(pH,qH))
                            modelH=ARIMA(dataH,(pH,dH,qH)).fit()

                            preA=modelA.forecast(1)   #預測未來1組，返回預測結果，標準差，置信區間        
                            preA=preA[0][0]
                            preH=modelH.forecast(1)   #預測未來1組，返回預測結果，標準差，置信區間        
                            preH=preH[0][0] 

                            d1=list(THA1.TIME)[-1]
                            d2=list(THA1.TIME)[-2] 
                            d3=datetime.datetime.timestamp(d1)+datetime.datetime.timestamp(d1)-datetime.datetime.timestamp(d2)
                            dateArray=time.localtime(d3)                #= datetime.datetime.utcfromtimestamp(d3)
                            therStyleTime=time.strftime('%Y-%m-%d %H:%M:%S',dateArray)      
                            preTIME.append(therStyleTime)

                            preAREA.append(preA)                   
                            preHIGHT.append(preH)   
                            predictTHA=[preTIME,preAREA,preHIGHT]
                            predictTHA=pd.DataFrame(predictTHA,index=['DateTime','CPKH','CPKA'])
                            predictTHA=predictTHA.T
                            predictTHA=predictTHA.drop_duplicates()
                            predictTHA.sort_index(by='DateTime',inplace=True)

                            fileName4='pre%s.txt'%pad[jj]
                            filePath4=file2+os.path.sep+fileName4                    
                            predictTHA.to_csv(filePath4,sep=',',index=False,header=None,mode='a+')  #,mode='a+'

                            with open(filePath3, 'r') as f3:
                                d3=f3.readlines()    
                                HIGHT3=[]            
                                AREA3=[]   
                                DateTime3=[]
                                for j3 in range(len(d3)):
                                    dt3=d3[j3].split(',')[0]
                                    ddt3=datetime.datetime.strptime(dt3,'%Y-%m-%d %H:%M:%S') 
                                    hight3=float(d3[j3].split(',')[1])
                                    area3=float(d3[j3].split(',')[2].split('\n')[0])   
                                    DateTime3.append(ddt3)
                                    HIGHT3.append(hight3)
                                    AREA3.append(area3)    
                              
                            with open(filePath4, 'r') as f4:
                                d4=f4.readlines()
                                HIGHT4=[]            
                                AREA4=[]   
                                DateTime4=[]                                    
                                for j4 in range(len(d4)):
                                    dt4=d4[j4].split(',')[0]
                                    ddt4=datetime.datetime.strptime(dt4,'%Y-%m-%d %H:%M:%S')  
                                    hight4=float(d4[j4].split(',')[1])
                                    area4=float(d4[j4].split(',')[2].split('\n')[0])   
                                    DateTime4.append(ddt4)
                                    HIGHT4.append(hight4)
                                    AREA4.append(area4)                  

                            if len(d3)>len(d4) and len(d4)>1:
                                for iii in range(len(d4)-1):
                                    DateTime4[-iii-2]=DateTime3[-iii-1]
                            elif len(d4)>len(d3) and len(d4)>1:
                                 for iiii in range(len(d3)):
                                    DateTime4[-iiii-2]=DateTime3[-iiii-1]

                            dd4=[DateTime4,HIGHT4,AREA4]
                            dd4=pd.DataFrame(dd4,index=['DateTime','HIGHT','AREA'])
                            db4=dd4.T                                  
                            db4.sort_values(by='DateTime',ascending=True,inplace=True)
                            fileName5='predict_%s.txt'%pad[jj]
                            filePath5=file2+os.path.sep+fileName5                    
                            db4.to_csv(filePath5,sep=',',index=False,header=None)   
                                              
os.system("pause")                     

In [2]:
int(2.8)

2